# Procesado e incorporación de datos de PIB per capita de los países de la UE
En este notebook se unen los datos de PIB per capita con el dataset global. 

In [2]:
import pandas as pd
import glob
import os

Primero, extraemos la información necesaria del excel y la guardamos en un csv.

In [3]:
ruta_carpeta = './Purchasing_power_data/'
df = pd.read_excel('./Purchasing_power_data/pib_capita.xlsx', sheet_name='Sheet 1', skiprows=7, nrows=28)

df.columns = df.iloc[0] 
df = df[1:]
df = df.loc[:, ~df.columns.isna()]
columnas_ordenadas = [df.columns[0]] + sorted(df.columns[1:], key=int)
df = df[columnas_ordenadas]
df.rename(columns={'TIME':'Country'}, inplace=True)
df.reset_index(drop=True, inplace=True) 

nombre_salida = 'pib_capita.csv'
ruta_salida = os.path.join(ruta_carpeta, nombre_salida)
df.to_csv(ruta_salida, index=False)

Ahora, juntamos la información de ese csv con su correspondiente csv de país. 

In [5]:
countries = ['austria','belgium','bulgaria','croatia','cyprus','czechia','denmark','estonia','finland','france','germany','greece','hungary',
             'ireland','italy','latvia','lithuania','luxembourg','malta','netherlands','poland','portugal','romania','slovenia','spain','sweden']

In [8]:
pib_data = pd.read_csv('./Purchasing_power_data/pib_capita.csv')

pib_long = pib_data.melt(id_vars=['Country'], var_name='Año', value_name='PIB')
pib_long['Año'] = pib_long['Año'].astype(str)  

for country in countries:
    country_data = pd.read_csv(f'./Countries_citricos_completos/{country.capitalize()}.csv')
    country_data['Año'] = country_data['Año'].astype(str)

    pib_country_data = pib_long[pib_long['Country'] == country.capitalize()]

    df_merged = country_data.merge(pib_country_data, on='Año', how='left')
    df_merged.drop(columns='Country', inplace=True)
    df_merged.rename(columns={'PIB':'PIB per capita'}, inplace=True)

    df_merged.to_csv(f'./Countries_citricos_completos/{country.capitalize()}.csv', index=False)
